In [ ]:
import os
from google.colab import drive

# setup
drive.mount('/content/drive')
root = '/content/drive/MyDrive/NeuroMagos_Checkpoints'
os.makedirs(root, exist_ok=True)

if not os.path.exists('NeuroMagos'):
    !git clone https://github.com/ChandraguptSharma07/NeuroMagos.git

%cd NeuroMagos

# data
fid = 'PASTE_FILE_ID_HERE'
if not os.path.exists('Synapse_Dataset'):
    !pip install -r requirements.txt
    !gdown --id {fid} -O data.zip
    !unzip -q data.zip

In [ ]:
# run
# uncomment resume arg to continue
last = os.path.join(root, 'last_model.pth')

!python train.py --epochs 100 --save_dir {root} # --resume {last}